In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [31]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

Loading data...


In [5]:
max([len(i) for i in x_train])

2494

In [7]:
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)

In [9]:
model = Sequential()
model.add(Embedding(20000, 128) )
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [11]:
model.build(input_shape=(None, 128))

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 128, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,691,713 (10.27 MB)

 Trainable params: 2,691,713 (10.27 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy' ])

In [43]:
model.fit(x_train, y_train,
batch_size=32,
epochs=1,
verbose=2,
validation_data=(x_test, y_test))

782/782 - 52s - 67ms/step - accuracy: 0.7833 - loss: 0.4617 - val_accuracy: 0.8337 - val_loss: 0.3802


In [44]:
score, acc = model.evaluate(x_test, y_test,
batch_size=32,
verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 - 10s - 13ms/step - accuracy: 0.8337 - loss: 0.3802
Test score: 0.38022807240486145
Test accuracy: 0.8336799740791321


In [48]:
test_review = "First: The recent campaigning of this movie is a huge hoax. Judging from the cover you'd think this was some kind of scandal movie about Kylie playing a character having sex, taking drugs and whatever. This is just a cheap market-scheme. She's barely in it and does neither of the things. The marketing here is unbelievable, and I'm surprised the filmmakers hasn't objected.The movie itself was to me a huge disappointment. It seemed like a Sunset Beach episode directed sloppy-handed by a teenage Quentin Tarantino. And thats not meant as a compliment, mind you.I think the weakness of the movie first of all is the story. It seems to be about nothing. Just about cool teenagers tripping around living 'on the edge'. The characters themselves does have some personality though, but the movie doesn't use its potential. As said, there's no story of any substance here. It seems to elaborate too much on cool dialog and ends up looking like a colorful MTV ad. It definitely has that feeling.Still though, I guess some people might enjoy it, but I'd say there's far better movies like this around."

In [49]:
def sentiment(faqs):
    tokenizer.fit_on_texts([faqs])
    input_sequences = []
    for sentences in faqs.split('.'):
        tokenized_sent = tokenizer.texts_to_sequences([sentences])[0]
    
        for i in range(1, len(tokenized_sent)):
            n_gram = tokenized_sent[:i+1]
            input_sequences.append(n_gram)
    padded_seq = sequence.pad_sequences(input_sequences, maxlen = 80, padding = 'pre')
    y = model.predict(padded_seq)
    if y[0]<0.5:
        print("According to the model, the sentiment of the review is positive")
    else:
        print("According to the model, the sentiment of the review is negative")    

In [50]:
sentiment(test_review)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
According to the model, the sentiment of the review is negative
